# Program mencoba model kelas 8 pake GoogleColab

sumber : https://neptune.ai/blog/predicting-stock-prices-using-machine-learning

In [9]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, LSTM
from keras.callbacks import ModelCheckpoint

In [10]:
# menghubungkan drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# mengimport file python/modul python yang ada di drive
import sys
sys.path.insert(0,'/content/drive/MyDrive/iky/fungsi')
import datanya

In [12]:
layer_units, optimizer = 50, 'adam' 
cur_epochs = 15
cur_batch_size = 20
window_size = 50 

# ~~~~~ file yang berisi ticker jadikan sebuah list
with open('drive/MyDrive/iky/data/ticker_issi.txt') as file:
    tick = file.readlines()
    ticker = [line.rstrip() for line in tick]# ini adalah list yang berisi ticker

In [13]:
ticker = ['YPAS']#'APEX']#,

In [14]:
for tic in ticker:
    # df = pd.read_csv(f'drive/MyDrive/iky/data/bsaham/{tic}.csv')
    df = pd.read_csv(f'drive/MyDrive/campuran/data/saham/{tic}.csv')
    X_train, y_train = datanya.data_latih12(df, window_size)
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True,input_shape=(X_train.shape[1],1)))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(LSTM(units=50))
    model.add(Dense(1, activation='linear'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    # filepath = f"drive/MyDrive/iky/kelas_8/model/{tic}_model.h5"
    # checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
    # callbacks_list = [checkpoint]
    # model.fit(X_train, y_train, batch_size=cur_batch_size, epochs=cur_epochs, callbacks=callbacks_list,
    #           shuffle=True)
    model.fit(X_train, y_train, batch_size=cur_batch_size, epochs=cur_epochs,
              shuffle=True)

Epoch 1/15
171/171 [==============================] - 17s 67ms/step - loss: 0.0048
Epoch 2/15
171/171 [==============================] - 12s 70ms/step - loss: 0.0017
Epoch 3/15
171/171 [==============================] - 11s 67ms/step - loss: 0.0015
Epoch 4/15
171/171 [==============================] - 12s 71ms/step - loss: 0.0012
Epoch 5/15
171/171 [==============================] - 11s 67ms/step - loss: 9.6125e-04
Epoch 6/15
171/171 [==============================] - 12s 70ms/step - loss: 8.3152e-04
Epoch 7/15
171/171 [==============================] - 11s 67ms/step - loss: 7.5549e-04
Epoch 8/15
171/171 [==============================] - 12s 68ms/step - loss: 7.1140e-04
Epoch 9/15
171/171 [==============================] - 12s 68ms/step - loss: 6.3319e-04
Epoch 10/15
171/171 [==============================] - 12s 68ms/step - loss: 6.0015e-04
Epoch 11/15
171/171 [==============================] - 11s 66ms/step - loss: 5.6520e-04
Epoch 12/15
171/171 [==============================] - 11

In [15]:
# ~~~~~~~~~~~~~~~~ fungsi untuk menggunakan model kelas 1 untuk memprediksi

import pandas as pd
from keras.models import load_model
from datetime import date

## fungsi untuk menghitung kenaikan harga
def kenaikan_persen(nilai_awal, nilai_terakhir):
    persen = float(((nilai_terakhir-nilai_awal)/nilai_awal)*100)
    if persen >= 0:
        akan = [1,'Naik']
    elif persen <= 0:
        akan = [0,'Turun']
    return persen, akan

def pake_k8(list_ticker, timestep=50):
    hasil_model = pd.DataFrame(columns=['Saham','Perubahan nilai','Keadaan','0/1','Perubahan(%)'])
    for tic in list_ticker:
        makanannya = pd.read_csv(f'drive/MyDrive/campuran/data/saham/{tic}.csv')
        hasil_pred = datanya.predik_5h(makanannya, timestep, model)
        kenaikan = hasil_pred[4][0] - hasil_pred[0][0]
        persentase, akan = kenaikan_persen(hasil_pred[0][0], hasil_pred[4][0])
        hasil_model = pd.concat([hasil_model, pd.DataFrame.from_records([{
            'Saham': tic, 'Perubahan nilai' : kenaikan, 'Keadaan': akan[1],
            '0/1':akan[0],'Perubahan(%)' : persentase}])])
    # mengurutkan sesuai kolom Perubahan(%), dari nilai terbesar ke terkecil
    hasil_model.sort_values(by='Perubahan(%)',ascending=False, inplace=True) 
    hasil_model.reset_index(inplace=True, drop=True) # mereset index
    return hasil_model

In [16]:
hasil_k8 = pake_k8(ticker)
hasil_k8

,Saham,Perubahan nilai,Keadaan,0/1,Perubahan(%)
0,YPAS,15.723896,Naik,1,3.099523
